# DEVM MultiNLI

In [1]:
import pandas as pd
import pickle

In [5]:
df =pd.read_pickle("../../../RIT_LLM/predictions/RL_MultiNLI_DEVM_600.pickle")
df

,relation,no_matcheadas,contradiction,jaccard,simBoW,sumas,mutinf_t,max_info_t,entail,list_comp,list_incomp,Jaro-Winkler_rit,negT,negH,overlap_ent,clases,sentence_A,sentence_B
0,2,0.333333,0.000000,0.333333,0.485071,0.163474,4.527890,0.850460,1.000000,1,0,0.650000,1,1,0.666667,entailment,"""So your girl writes that your little farewel...",Your farewell activity didn't go well.
1,2,0.200000,0.000000,1.000000,0.615457,0.149428,4.527443,0.879779,1.000000,1,0,0.590909,0,0,0.800000,entailment,"Under Deng Xiaoping, Beijing actively sought t...",Beijing sought to create a good relationship.
2,2,0.250000,0.000000,0.500000,0.535303,0.038836,4.411816,0.731585,0.888889,2,0,0.497076,0,0,0.750000,entailment,GAO also issued over 160 reports detailing spe...,Improving government readiness was aided by th...
3,1,0.000000,0.000000,1.000000,1.000000,0.340767,4.654967,1.000000,1.000000,0,0,1.000000,0,0,1.000000,entailment,The chain wielder smiled at her.,The chain wielder was smiling.
4,2,0.600000,0.000000,0.500000,0.353553,0.075418,4.190316,0.550815,0.800000,2,0,0.563889,0,0,0.400000,entailment,The celebrity-obsessed magazine surpasses itse...,The magazine publishes a lot of stories on cel...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,2,1.000000,0.000000,0.000000,0.198030,0.011191,3.987838,0.441345,0.600000,2,0,0.408497,0,1,0.000000,contradiction,There are also a couple of small aircraft lyin...,Abandoned aircraft have never been found in th...
596,2,1.000000,0.000000,0.000000,0.182574,0.090118,3.795031,0.510137,1.000000,1,0,0.000000,0,0,0.000000,contradiction,"""The summons was only for Dave Hanson,"" Ser P...",All of them had been invited.
597,-1,0.333333,0.333333,0.333333,0.306186,0.147137,4.739438,1.000000,1.000000,0,0,0.000000,0,0,0.333333,contradiction,The analysis also addresses the various altern...,There are no standards for design.
598,1,0.000000,0.000000,0.666667,0.730297,0.047815,4.365051,0.698024,0.600000,0,0,0.822222,0,0,1.000000,contradiction,Scotland became little more than an English co...,Scotland was far greater than an English county.


# Abrir archivos generados y validar que esten completos

## DEVM

In [6]:
lista_respuestas=[]

In [9]:
# agarramos el ultimo por que tiene a todos solo hay que revisar los faltantes para incorporarlos
with open("rit_MultiNLI_DEVM_600_.pickle", "rb") as f:
    lista_respuestas = pickle.load(f)
# Imprime [1, 2, 3, 4].
print(len(lista_respuestas))

600


In [10]:
# Validar si faltan esos tres
i=0
for a in lista_respuestas:
    if a=="NA":
        print(i)
    i+=1

304


In [15]:
lista_respuestas[304]='{\n  "answer": "Entailment"\n}'

In [6]:
faltantes=[2,4,37,66,67,68,69,70,71,72,73,74,75,76,77,92,98,119,138,203,366]

# obtener los faltantes

In [7]:
import requests
import json
import random
import time

model = "phi3" # seleeciona el modelo a usar

template = {
  "Answer": "",
  "Score Lexical Semantics": "",
  "Score Predicate-Argument Structure": "",
  "Score Logic": "",
  "Score Knowledge & Common Sense": ""
}


In [8]:
faltantes

[2,
 4,
 37,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 92,
 98,
 119,
 138,
 203,
 366]

In [9]:
faltantes_respuestasOllama=[]
for i in range(1):
    for index,strings in df.iterrows():
        if index in faltantes:
            prompt = ''''You are a experte in Recognition Textual Entailment. Analyze the following sentences: 
            Text: '''+strings["sentence_A"]+'''
            Hypothesis: '''+ strings["sentence_B"]+'''   
            only responds using the template. '''
            #print(prompt)
            data = {
                "prompt": prompt,
                "model": model,
                "format": "json",
                "stream": False,
                #"options": {"temperature": 0.2},
            }
            try:
                response = requests.post("http://localhost:11434/api/generate", json=data, stream=False,timeout=15)
                json_data = json.loads(response.text)
                faltantes_respuestasOllama.append(json.dumps(json.loads(json_data["response"]), indent=2))
                print(index)
            except:
                print("Saltó",index)
                faltantes_respuestasOllama.append("NA")

2
4
37
66
67
68
69
70
71
72
73
74
75
76
77
92
98
119
138
203
366


In [10]:
faltantes_respuestasOllama

['{\n  "Explanation": "Yes",\n  "Answer": "The hypothesis that improving government year 2000 conversion readiness was in part due to efforts from the General Accountability Office (GAO), who issued a significant number of reports, is entailed by the text. As indicated by the original statement\'s mentioning over \'160 reports detailing specific findings and more than 100 recommendations\', it supports that the GAO did make considerable efforts to enhance government readiness."\n}',
 '{\n  "input_text": "The celebrity-obsessed magazine surpasses itself in the post-Oscar issue.",\n  "output_hypothesis": "Yes"\n}',
 '{\n  "entails": true,\n  "neutral": false,\n  "contradicts": false\n}',
 '{\n  "entails": true\n}',
 '{\n  "claim": "Treasure Beach",\n  "type": "neutral"\n}',
 '{\n  "entails": false,\n  "reasoning": "The text implies that all of the \'dirty\' tasks were completed in the past but does not suggest that there is none remaining."\n}',
 '{\n  "entails": true,\n  "reasoning": "T

In [12]:
print(faltantes)
faltantes_respuestasOllama

[312, 442, 575]


['{\n  "Answer": "Implication",\n  "Score of Lexical Semantics (0-1)": "0.6",\n  "Score of Predicate-Argument Structure (0-1)": "0.75",\n  "Score of Logic (0-1)": "0.8",\n  "Score of Knowledge & Common Sense (0-1)": "0.4"\n}',
 '{\n  "Answer": "Neutral",\n  "Score of Lexical Semantics (0-1)": "Low - The words used in both sentences are simple and common, with no complex or specialized vocabulary.",\n  "Score of Predicate-Argument Structure  (0-1)": "Medium - Both the text and hypothesis have clear subjects (\'i\' and \'we\') performing actions. However, there is a mismatch in verb forms between them.",\n  "Score of Logic (0-1)": "Low - The two sentences do not logically connect or contradict each other.",\n  "Score of Knowledge & Common Sense (0-1)": "Medium - While the text and hypothesis are both everyday expressions, they don\'t share a common context.",\n  "sums": 0.10576550783589482,\n  "negH": 0,\n  "list_compability": 4,\n  "list_incompability": 0,\n  "relation": 2,\n  "entail":

In [13]:
#Mezclar faltantes 
j=0
for i in faltantes:
    lista_respuestas[i]=faltantes_respuestasOllama[j]
    j+=1

In [14]:
len(lista_respuestas)

600

# Crear una dataframe con las respuestas de OLLAMA

In [11]:
import json

In [16]:
new_data = {'prediction':[]}#, 'score_LS':[], 'score_PAS':[], 'score_L':[], 'score_K':[]}

In [17]:
lista_respuestas

['{\n  "answer": "Entailment"\n}',
 '{\n  "label": "Entailment",\n  "explanation": "The hypothesis that \'Beijing sought to create a good bilateral relationship\' is entailed by the text, as it specifically mentions Beijing seeking a good bilateral (two-country) relationship under Deng Xiaoping."\n}',
 '{\n  "answer": "Entailment"\n}',
 '{\n  "label": "Entailment",\n  "explanrances": [\n    "The act of the chain wielder smiling is mentioned in both text and hypothesis, so this shows entailment."\n  ]\n}',
 '{\n  "label": "Entailment",\n  "explanent": "The celebrity-obsessed magazine typically includes numerous articles focusing on various aspects and lives of celebrities, which is consistent with its self-description as surpassing itself in the post-Oscar issue."\n}',
 '{\n  "label": "Entailment",\n  "explanation": "The text clearly states that the person usually goes for vegetables but makes an exception when they\'re having french fries, which aligns with the hypothesis provided."\n}

In [18]:
k=0
for a in lista_respuestas:
    temp_dict=json.loads(a)
    # if (len(list(temp_dict.keys()))<5):
    #     print("indice",k)
    # k+=1
    list_keys=list(temp_dict.keys())
    new_data['prediction'].append(temp_dict[list_keys[0]])
    # new_data['score_LS'].append(temp_dict[list_keys[1]])
    # new_data['score_PAS'].append(temp_dict[list_keys[2]])
    # new_data['score_L'].append(temp_dict[list_keys[3]])
    # new_data['score_K'].append(temp_dict[list_keys[4]])


In [19]:
df_new_data=pd.DataFrame(new_data)
df_new_data

,prediction
0,Entailment
1,Entailment
2,Entailment
3,Entailment
4,Entailment
...,...
595,Contradiction
596,Neutral
597,Neutral
598,Contradiction


In [20]:
df_new_data.to_pickle("../finales/MultiNLI_DEVM_dRIT.pickle")

In [39]:
for index, strings in df_new_data.iterrows():
    if (type(strings["prediction"])==type(True)):
        print(index)

191
397
483
487


In [36]:
df_new_data["prediction"]

0         Entailment
1         Entailment
2         Entailment
3         Entailment
4         Entailment
           ...      
595    Contradiction
596          Neutral
597          Neutral
598    Contradiction
599    Contradiction
Name: prediction, Length: 600, dtype: object

In [40]:
def convertirLabel(l):
    new_l=[]
    for a in l:
        if (type(a)==type("a")):
            if a.upper() == "IMPLICATION" or  a.upper() == "ENTAILMENT" or  a.upper() == "ENTAILED":
                new_l.append(1)
            elif a.upper() == "NEUTRAL":
                new_l.append(2)
            else:
                new_l.append(0)
        else:
            new_l.append(0)
    return new_l

In [41]:
y_true=convertirLabel(df["clases"].values)
y_pred=convertirLabel(df_new_data["prediction"].values)

In [42]:
from sklearn.metrics import confusion_matrix,classification_report

In [43]:
print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.80      0.84       200
           1       0.86      0.81      0.84       200
           2       0.68      0.79      0.73       200

    accuracy                           0.80       600
   macro avg       0.81      0.80      0.80       600
weighted avg       0.81      0.80      0.80       600

[[159   1  40]
 [  4 163  33]
 [ 17  25 158]]
